In [1]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup
from selenium import webdriver

In [25]:
nasa_url = "https://mars.nasa.gov/news/"
twitter_url = "https://twitter.com/marswxreport?lang=en"
facts_url = "http://space-facts.com/mars/"
astro_base = 'https://astrogeology.usgs.gov/search/'
astro_url = astro_base+"results?q=hemisphere+enhanced&k1=target&v1=Mars"
url_list = [nasa_url,twitter_url,facts_url,astro_url]

In [3]:
soups = {}
driver = webdriver.Firefox()
for url in url_list:
    driver.get(url)
    driver.implicitly_wait(20)
    soups.update({url: BeautifulSoup(driver.page_source,"lxml")})
driver.close()

In [4]:
nasa_latest = {'title':soups[nasa_url].body.find('div',class_='content_title').text,
                'teaser':soups[nasa_url].body.find('div',class_='article_teaser_body').text}
nasa_latest

{'title': 'The MarCO Mission Comes to an End',
 'teaser': 'The pair of briefcase-sized satellites made history when they sailed past Mars in 2019.'}

In [36]:
driver = webdriver.Firefox()
driver.get(twitter_url)
driver.implicitly_wait(10)
soups.update({url: driver.page_source})
with open('twitter.text','w+') as fp:
    fp.write(driver.page_source)
driver.close()

In [25]:
twit = soups[twitter_url].body.find_all('span')
len(twit)

6

In [15]:
facts_df = pd.read_html(str(soups[facts_url].body.find('table',id='tablepress-p-mars-no-2')))[0].set_index(0)
(facts_df)

,1
0,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [21]:
astro_a = soups[astro_url].body.find_all('a',class_='itemLink product-item')
astro_links = [ a['href'] for a in astro_a]
astro_links

['/search/map/Mars/Viking/cerberus_enhanced',
 '/search/map/Mars/Viking/cerberus_enhanced',
 '/search/map/Mars/Viking/schiaparelli_enhanced',
 '/search/map/Mars/Viking/schiaparelli_enhanced',
 '/search/map/Mars/Viking/syrtis_major_enhanced',
 '/search/map/Mars/Viking/syrtis_major_enhanced',
 '/search/map/Mars/Viking/valles_marineris_enhanced',
 '/search/map/Mars/Viking/valles_marineris_enhanced']

In [44]:
astro_img_url = []
[astro_img_url.append(astro_base+a) for a in astro_links if astro_base+a not in astro_img_url]
astro_img_url

['https://astrogeology.usgs.gov/search//search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search//search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov/search//search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search//search/map/Mars/Viking/valles_marineris_enhanced']